# Downloading Dataset

In [ ]:
!pip install --upgrade --no-cache-dir gdown

In [ ]:
!gdown https://drive.google.com/uc?id=1mLkyvMB7EBbMpAA4f_uLBxOr34U84miD
!unzip -q retail_4_classes.zip

# Imports

In [ ]:
import torch
import matplotlib.pyplot as plt
import numpy as np

# Dataset Preparation

Note: you can also create your own dataset wrapper like in the previous lecture

https://colab.research.google.com/drive/1-uxaA8ZhpgLtctCKrzm2tF_U-_aeF_ci#scrollTo=nZld2EsoQkn9

In [ ]:
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchvision import transforms

preprocessing = transforms.Compose([
   transforms.Resize([224,224]),   
   transforms.RandomRotation(5),                             
   transforms.ToTensor(), 
])

trainset = ImageFolder('retail_4_classes/train', transform = preprocessing)
dataloader = DataLoader(dataset=trainset, batch_size=16, 
                        shuffle=True)

## Test reading the dataset

In [ ]:
img, label = trainset[100]

print(img.shape)
print(label)
image = np.transpose(img, [1,2,0])

plt.imshow(image)

## Test the dataset loader

In [ ]:
for i, (x,y) in enumerate(dataloader):
    print(x.shape, y.shape)
    break

In [ ]:
idx = 5
image = np.transpose(x[idx].cpu().numpy(), [1,2,0])
print(y[idx])
plt.imshow(image)

# Create Model from Pre-trained Architecture

## Install timm
https://github.com/rwightman/pytorch-image-models

In [ ]:
!pip install timm

## Creating the model

Check https://github.com/rwightman/pytorch-image-models#models for details of the models

In [ ]:
import torch
import torch.nn as nn

import timm

class Net(nn.Module):
    def __init__(self, n_class, pretrained=True):
        super(Net, self).__init__()

        self.backbone = timm.create_model(model_name='efficientnet_b0', num_classes=n_class, pretrained=pretrained)
        
    def forward(self, x):
        out = self.backbone(x)
            
        return out


# Testing the model

In [ ]:
model = Net(4)
pred = model(x)

print(pred.shape)
print(len(pred.shape))

# Training Utilities

In [ ]:
from torch.nn.functional import cross_entropy

def training(model, data_loader, objective_function, device, optimizer, max_iter=-1):
    model.train()

    for i, (x,y) in enumerate(data_loader):
        # moves the trainig data to device (possibly GPU)
        x = x.to(device)
        y = y.to(device)

        # inference
        logits = model(x)

        # calculate the error score
        loss = objective_function(logits, y) # menggunakan cross_entropy

        # updating the parameters
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if i==max_iter:
            break

In [ ]:

cpu = torch.device('cpu')
def count_acc(logits, label, device = cpu):
    pred = torch.argmax(logits, dim=1)
    return (pred == label).to(device).type(torch.FloatTensor).mean().item()

@torch.no_grad()
def evaluation(model, data_loader, objective_function, device, max_iter=-1):
    model.eval()

    loss_values = []
    accuracy_values = []

    for i, (x,y) in enumerate(data_loader):
        # moves the trainig data to device (possibly GPU)
        x = x.to(device)
        y = y.to(device)

        # inference
        logits = model(x)

        # calculate the error score
        loss = objective_function(logits, y)
        acc = count_acc(logits, y, device)

        loss_values.append(loss.item())
        accuracy_values.append(acc)

        if i==max_iter:
            break

    # calculate the average value of loss and accuracy
    loss = np.mean(loss_values)
    accuracy = np.mean(accuracy_values)

    return loss, accuracy

# Training

## Data Preparation

In [ ]:
batch_size = 16

'''training data'''
trainset = ImageFolder('retail_4_classes/train', transform = preprocessing)
train_loader = DataLoader(dataset=trainset, batch_size=batch_size, 
                        shuffle=True)

'''validation data'''
val_preprocessing = transforms.Compose([
   transforms.Resize([224,224]),                            
   transforms.ToTensor(), 
])
valset = ImageFolder('retail_4_classes/val', transform = val_preprocessing)
val_loader = DataLoader(dataset=valset, batch_size=batch_size, 
                        shuffle=False)

## Define the model (training from scratch)

In [ ]:
model = Net(4, pretrained=False)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

## Training Process

In [ ]:
epochs = 5
model_checkpoint = 'best_model.pth'

best_acc = 0
for e in range(epochs):
    training(model, train_loader, cross_entropy, device, optimizer)
    loss, acc = evaluation(model, val_loader, cross_entropy, device)
    scheduler.step()

    print('epoch %d loss=%f acc=%f'%(e, loss, acc))
    
    if acc>best_acc:
        torch.save(model.state_dict(), model_checkpoint)
        best_acc = acc

## Define the model (fine tuning)

In [ ]:
model = Net(4, pretrained=True)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

## Training Process

In [ ]:
epochs = 5
model_checkpoint = 'best_model.pth'

best_acc = 0
for e in range(epochs):
    training(model, train_loader, cross_entropy, device, optimizer)
    loss, acc = evaluation(model, val_loader, cross_entropy, device)
    scheduler.step()

    print('epoch %d loss=%f acc=%f'%(e, loss, acc))
    
    if acc>best_acc:
        torch.save(model.state_dict(), model_checkpoint)
        best_acc = acc

# Loading the trained model and testing it

In [ ]:
model = Net(4)

model.load_state_dict(torch.load(model_checkpoint))
model.to(device)

'''validation data'''
valset = ImageFolder('retail_4_classes/val', transform = val_preprocessing)
val_loader = DataLoader(dataset=valset, batch_size=1, 
                        shuffle=False)

loss, acc = evaluation(model, val_loader, cross_entropy, device, max_iter=-1)
print('val:', loss, acc)

# Inference function

In [ ]:
@torch.no_grad()
def predict(model, x, device=cpu):
    model.eval()

    logits = model(x.to(device))
    pred = torch.argmax(logits, dim=1)
    return pred

In [ ]:
for i, (x,y) in enumerate(dataloader):
    print(x.shape, y.shape)
    break

prediction = predict(model, x, device)

print('preds:', prediction)
print('label:', y)

# Transfer Learning Model

In [ ]:
import torch
import torch.nn as nn

import timm

class NetTransferLearning(nn.Module):
    def __init__(self, n_class, pretrained=True, model_name='efficientnet_b0', feat_size=1280):
        super(NetTransferLearning, self).__init__()

        self.backbone = timm.create_model(model_name=model_name, num_classes=n_class, pretrained=pretrained)
        self.backbone._modules['classifier'] = nn.Identity()

        self.predictor = nn.Linear(in_features=feat_size, out_features=n_class, bias=True)

    def forward(self, x):
        feat = self.backbone(x)
        out = self.predictor(feat)
        return out


## Define the model (transfer learning)

In [ ]:
model = NetTransferLearning(4, pretrained=True)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

optimizer = torch.optim.Adam(model.predictor.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

In [ ]:
print(model.backbone._modules['conv_stem'].weight[0,0])
print(model.predictor.weight)

## Training Process

In [ ]:
epochs = 5
model_checkpoint = 'best_model.pth'

best_acc = 0
for e in range(epochs):
    training(model, train_loader, cross_entropy, device, optimizer)
    loss, acc = evaluation(model, val_loader, cross_entropy, device)
    scheduler.step()

    print('epoch %d loss=%f acc=%f'%(e, loss, acc))
    
    if acc>best_acc:
        torch.save(model.state_dict(), model_checkpoint)
        best_acc = acc

# Loading the trained model and testing it

In [ ]:
model.load_state_dict(torch.load(model_checkpoint))
model.to(device)

'''validation data'''
valset = ImageFolder('retail_4_classes/val', transform = val_preprocessing)
val_loader = DataLoader(dataset=valset, batch_size=1, 
                        shuffle=False)

loss, acc = evaluation(model, val_loader, cross_entropy, device, max_iter=-1)
print('val:', loss, acc)

In [ ]:
print(model.backbone._modules['conv_stem'].weight[0,0])
print(model.predictor.weight)